In [14]:
import tensorflow as tf

In [15]:
#from tensorflow.examples.tutorials.mnist import input_data

In [41]:
#mnist = input_data.read_data_sets("/tmp/data2",one_hot="TRUE")
#from glob import glob
import os
import glob
#glob.glob('./[0-9].*')
def read_data_sets(train_dir,
                   fake_data=False,
                   one_hot=False,
                   dtype=dtypes.float32,
                   reshape=True,
                   validation_size=5000):
  if fake_data:

    def fake():
      return DataSet([], [], fake_data=True, one_hot=one_hot, dtype=dtype)

    train = fake()
    validation = fake()
    test = fake()
    return base.Datasets(train=train, validation=validation, test=test)

  TRAIN_IMAGES = 'train-images-idx3-ubyte.gz'
  TRAIN_LABELS = 'train-labels-idx1-ubyte.gz'
  TEST_IMAGES = 't10k-images-idx3-ubyte.gz'
  TEST_LABELS = 't10k-labels-idx1-ubyte.gz'

  local_file = base.maybe_download(TRAIN_IMAGES, train_dir,
                                   SOURCE_URL + TRAIN_IMAGES)
  with open(local_file, 'rb') as f:
    train_images = extract_images(f)

  local_file = base.maybe_download(TRAIN_LABELS, train_dir,
                                   SOURCE_URL + TRAIN_LABELS)
  with open(local_file, 'rb') as f:
    train_labels = extract_labels(f, one_hot=one_hot)

  local_file = base.maybe_download(TEST_IMAGES, train_dir,
                                   SOURCE_URL + TEST_IMAGES)
  with open(local_file, 'rb') as f:
    test_images = extract_images(f)

  local_file = base.maybe_download(TEST_LABELS, train_dir,
                                   SOURCE_URL + TEST_LABELS)
  with open(local_file, 'rb') as f:
    test_labels = extract_labels(f, one_hot=one_hot)

  if not 0 <= validation_size <= len(train_images):
    raise ValueError(
        'Validation size should be between 0 and {}. Received: {}.'
        .format(len(train_images), validation_size))

  validation_images = train_images[:validation_size]

  validation_labels = train_labels[:validation_size]
  train_images = train_images[validation_size:]
  train_labels = train_labels[validation_size:]

  train = DataSet(train_images, train_labels, dtype=dtype, reshape=reshape)
  validation = DataSet(validation_images,
                       validation_labels,
                       dtype=dtype,
                       reshape=reshape)
  test = DataSet(test_images, test_labels, dtype=dtype, reshape=reshape)

  return base.Datasets(train=train, validation=validation, test=test)



type_1_files = glob.glob(os.path.join("/tmp/data2/Cervix","*.jpg"))
print len(type_1_files)
#import cv2
#cv2.imshow(type_1_files[0])

512


In [42]:
""" Set up the computation graph in tensor flow by
    constructing a deep neural network with 3 hidden layers
    Feed-forward network: (Forward pass)
        input (x) --> hidden layer1 --> hidden layer2 --> hidden layer3 --> output
    Backpropagation:
        compare model's predicted output with expected output(y) -> cost function (cross entropy)
        Optimization function (optimizer) -> minimizes cost (AdamOptimizer, AdaGrad)
        Backward pass --> To minimize cost, weights of hidden layers are manipulated
        in the backward pass        
    Batch size: 
        Number of training examples in one forward/backward pass, it takes memory
    Epoch:
        Forward + backward pass of all the training examples.
    Iteration:
        one pass = forward pass + backward pass
        one iteration = number of passes, each pass using [batch size] training examples.
"""
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
n_classes = 10
batch_size = 100 
num_epochs = 10
input_data_size = 784

In [43]:
""" place holder for input and output values """
x = tf.placeholder('float',[None,input_data_size])
y = tf.placeholder('float')

In [44]:

""" Model the neural network"""
def neural_network_model(data):
    
    """output = (input_data * weights) + biases
	
	W has a shape of [784, 10] because we want
	to multiply the 784-dimensional image vectors by it 
	to produce 10-dimensional vectors of evidence for the difference classes.
	
	b has a shape of [10] so we can add it to the output."""
    
    hidden_layer_1 = {"weights":tf.Variable(tf.random_normal([input_data_size,n_nodes_hl1])),
                      "biases":tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_layer_2 = {"weights":tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])),
                      "biases":tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_layer_3 = {"weights":tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
                      "biases":tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer = {"weights":tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
                      "biases":tf.Variable(tf.random_normal([n_classes]))}
    
    """ For each neuron in hidden layer:
        output = activation(sum((input*weight)) + biases)
        Hidden layer 1 takes actual input data, add it's weight
        and biases, and send the output to second layer and so on. 
        Output of hidden layer 3 will be fed to output layer 
		relu - Activation funtion, Computes rectified linear: max(features, 0)
    """
    output_hl1 = tf.add(tf.matmul(data,hidden_layer_1['weights']),hidden_layer_1['biases'])
    output_hl1 = tf.nn.relu(output_hl1)
    
    output_hl2 = tf.add(tf.matmul(output_hl1,hidden_layer_2['weights']),hidden_layer_2['biases'])
    output_hl2 = tf.nn.relu(output_hl2)
    
    output_hl3 = tf.add(tf.matmul(output_hl2,hidden_layer_3['weights']),hidden_layer_3['biases'])
    output_hl3 = tf.nn.relu(output_hl3)
    
    """ For each neuron in output layer:
        output = (input*weight) + biases"""
    output = tf.add(tf.matmul(output_hl3,output_layer['weights']),output_layer['biases'])
    
    return output
    

In [45]:
def train_neural_netwrok(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        """ Training the netwrok to optimize the weights """
        for epoch in range(num_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                _,c = sess.run([optimizer,cost], feed_dict={x:epoch_x,y:epoch_y})
                epoch_loss += c
            print "Epoch %s completed out of %s, with epoch_loss %s" %(epoch,num_epochs,epoch_loss)
        
        """ 
        After optimizing weights, run them through our model,
        and compare the prediction to actual label, and evaluate the
        accuracy of all the test data
        """
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print "Accuracy : %s" %(accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))

train_neural_netwrok(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 10, with epoch_loss 2229932.7254
Epoch 1 completed out of 10, with epoch_loss 442999.648571
Epoch 2 completed out of 10, with epoch_loss 237133.868792
Epoch 3 completed out of 10, with epoch_loss 141925.425902
Epoch 4 completed out of 10, with epoch_loss 87421.2980564
Epoch 5 completed out of 10, with epoch_loss 53707.4115194
Epoch 6 completed out of 10, with epoch_loss 37381.1915681
Epoch 7 completed out of 10, with epoch_loss 27981.0554463
Epoch 8 completed out of 10, with epoch_loss 20980.2657408
Epoch 9 completed out of 10, with epoch_loss 20402.8236441
Accuracy : 0.9503
